In [ ]:
import pandas as pd
from src.data_stats import compute_headline_lengths, headline_word_counts
from src.publisher_analysis import count_publishers, extract_top_keywords
from src.time_analysis import enrich_time_features, get_daily_article_counts, get_hourly_distribution
from src.publisher_analysis import extract_top_keywords
from src.visualize import plot_wordcloud

# Load data
df = pd.read_csv("../data/raw/news_data.csv")

# Descriptive statistics
print("Headline Lengths:\n", compute_headline_lengths(df))
print("Word Counts:\n", headline_word_counts(df))

# Publisher Analysis
print("Top Publishers:\n", count_publishers(df))
print("Top Keywords:\n", extract_top_keywords(df))

# Time Features
df = enrich_time_features(df)
print("Daily Article Counts:\n", get_daily_article_counts(df).head())
print("Hourly Distribution:\n", get_hourly_distribution(df).head())
# text_analysis
top_keywords = extract_top_keywords(df, column='headline', max_features=20)
print("🔑 Top Keywords in Headlines:", top_keywords)

# Visualizations
plot_wordcloud(df)

ModuleNotFoundError: No module named 'src'